In [2]:
%run "./Setuper.ipynb"

### Observer

In [18]:
from tqdm.notebook import trange
import random
import sys
import numpy as np
import chipwhisperer as cw

class ANN_Observer:
    project = None
    epochs = 0
    n_inputs = 7
    waves = None
    project_path = None
    prev_rand_inputs = None
    akt_rand_inputs = None
    out_data = None   
    cw_setuper : CW_Setuper = None
    trace_len = 24431
    akt_out_data = None
    output_database = []
    
    def __init__(self, _name, _epochs = 300, _cw_setuper : CW_Setuper = None, create = False, re_init_config = None, firmware_reinit = True, targ_periodic_reset = False):
        self.targ_periodic_reset = targ_periodic_reset
        self.epochs = _epochs
        self.akt_rand_inputs = None
        if _cw_setuper is None:
            self.cw_setuper = CW_Setuper(firmware_reinit = firmware_reinit, re_init_dict  = re_init_config)
        self.scope_setup(samples=24431, decimate=2)
        try:
            if create:
                self.load_project(_name = f"projects/{_name}")
                print("✔️ Project succesfully loaded.")
            else:
                raise Exception("Choosen aproach was to create new project, sorry..") 
        except:
            print("⚠️ The mentioned lab was not found!!!")
            print("🕓 Initiating self-creation")
            self.create_project(project_name = _name)
            print("✔️ Project succesfully created.")
        
    def ___create_project(self, project_name = "lab1"): #create_X_traces_with_rand_Nth_bit
        # Create an repositary/save point/project, and get ready chipwhisperer
        project = cw.create_project(f"projects/{project_name}", overwrite = True)
        self.project = project
        try:
            for i in trange(self.n_inputs, desc='Initiating creation of first layer sets of traces'):
                self.measure_traces(bnum = 8, input_size = self.n_inputs, rand_indx = i, rand_input = self.akt_rand_inputs)
            self.measure_traces(bnum = 8, input_size = self.n_inputs, rand_indx = 0, rand_input = np.zeros(self.epochs), change_rand = False)
        except Exception as e:
            print("❌ Failed measurements because: ",e)
            print("⚠️ Proceeding to disconect!!!")
            self._scope_disc_()

    def create_project(self, project_name = "lab1"): #create_X_traces_with_rand_Nth_bit
        # Create an repositary/save point/project, and get ready chipwhisperer
        try:
            project = cw.create_project(f"projects/{project_name}", overwrite = True)
            self.project = project
            self.measure_traces(bnum = 8, input_size = self.n_inputs, rand_indx = 0, rand_input = self.akt_rand_inputs)
        except Exception as e:
            print("❌ Failed measurements because: ",e)
            print("⚠️ Proceeding to disconect!!!")
            self._scope_disc_()
        
    def load_project(self, _name = "./trace.cwp", load = True):
        self.project = cw.open_project(f'projects/{_name}.cwp')
        return self.project

    def _scope_disc_(self):
        try:
            self.cw_setuper.scope.dis()
            self.cw_setuper.target.dis()
            print("✔️ Disconection succesfull.")
        except Exception as e:
            print("❌ Disconection failed because: ",e)
    
    def scope_setup(self, samples=24431, decimate=2):
        # arm the scope
        scope = self.cw_setuper.scope
        scope.arm()
        # Set the maximum number of points in a trace
        scope.adc.fifo_fill_mode = "normal"
        scope.adc.samples = samples
        scope.adc.decimate = decimate
        
    def capture_trace(self, input_data, cmd = 'p', print_ack = False):
        scope = self.cw_setuper.scope
        target = self.cw_setuper.target
        #Initial setup before capturing
        scope.arm() # arm the scope
        target.flush() # flush the UART buffer, normaly it is automatized, but in this case it is called manualy, it in essence flushes buffer storing incoming data t odevice
            
        #Send the data and capture
        target.send_cmd('p', 0x80, input_data)
        ret = scope.capture()
        wave = scope.get_last_trace()
        out_data = target.read_cmd('r') 
        if self.akt_out_data is None:
            self.akt_out_data = out_data 
        out_data = None
        ack = target.read_cmd('e')
        trace = cw.Trace(wave, input_data, out_data, []) #save the metadata
        if print_ack:
            print(ack)
            print(out_data)  
        out_data = None
        
        return trace

    def measure_traces(self, bnum = 8, input_size = 8, rand_indx = 0, rand_input = None, change_rand = False): #create_X_traces_with_rand_Nth_bit 
        #print(f"✔️ Starting measurement for {rand_indx}-th weights..")
        scope = self.cw_setuper.scope
        target = self.cw_setuper.target
        project = self.project
        self.cw_setuper.reset_target(scope) # flush chipwhisperer and re-assing its parameters (this is moot, if used >1x)
        rand_inputs = rand_input
        if rand_input is None:
            print(f"⚠️ Generating rand_input..")
            L = range((2**bnum)-1) # Range of generated rand_numbers
            rand_inputs = [random.choice(L) for _ in range(self.epochs)]
            self.akt_rand_inputs = rand_inputs
        input_array = [np.uint8(0) for i in range(input_size)]
        
        # capture X (epochs) traces, while each has different rand_input
        for i in trange(self.epochs, desc='                                Capturing traces'):
            input_array[rand_indx] = np.uint8(rand_inputs[i])
            input_data = bytearray(input_array)
            trace = self.capture_trace(input_data=input_data)
            project.traces.append(trace)
        
            #Send every 50th trace refress
            if self.targ_periodic_reset:
                if i % 60 == 0:
                    self.cw_setuper.reset_target(scope)
        if change_rand:
            self.prev_rand_inputs = self.akt_rand_inputs
            self.akt_rand_inputs = akt_rand_inputs
        project.save()
        self.project = project
        return project, out_data

    def normalize_trace(self, wave, treshold =  0.15):
        lenght =  len(wave)
        new_wave = np.ones((lenght,), dtype=int)
        for indx  in range(lenght):
            if abs(wave[indx]) < treshold:
                new_wave[indx] = 0
        return new_wave
    # wave  = normalize_trace(project.traces[0][0], treshold = 0.05)
    # plotSaveWave(wave = wave, name_wave = "temp_wave")
    
    def uppSampleArray(self, origin_row, degree): # uppsample by  padding after every time sample
        exp_array = []
        for i in origin_row:
            exp_array.append(i)
            for ii in range (degree):
                exp_array.append(0)
        return exp_array
        
    def createDiffWave(self):
        project = self.project
        waves = []
        diff_waves = []
        square_waves = []
        
        for trace in project.traces:
            waves.append(trace[0])
        avg_wave = np.mean(waves,axis=0)
        
        for i in waves:
            diff_waves.append(np.subtract(i,avg_wave))
        for i in diff_waves:
            square_wave = []
            for ii in i:
                square_wave.append(ii**2)
            square_waves.append(square_wave)
        
        diff_avg = np.mean(diff_waves,axis=0)
        var = np.mean(square_waves,axis=0)
    
        
        return diff_avg, avg_wave, var, waves

### General functions

In [4]:
def scope_setup(samples=24431, decimate=2):
        # arm the scope
        scope.arm()
        # Set the maximum number of points in a trace
        scope.adc.fifo_fill_mode = "normal"
        scope.adc.samples = samples
        scope.adc.decimate = decimate
def capture_trace(input_data, cmd = 'p', print_ack = False):
        #Initial setup before capturing
        scope.arm() # arm the scope
        target.flush() # flush the UART buffer, normaly it is automatized, but in this case it is called manualy, it in essence flushes buffer storing incoming data t odevice
            
        #Send the data and capture
        target.send_cmd(cmd, 0x80, input_data)
        ret = scope.capture()
        wave = scope.get_last_trace()
        out_data = target.read_cmd('r')
        ack = target.read_cmd('e')
        trace = cw.Trace(wave, input_data, out_data, []) #save the metadata
        if ret:
                print("Target timed out!")
        if print_ack:
            print(ack)
            print(out_data)  
        out_data = None
        
        return trace
def measure_traces(bnum = 8, input_size = 8, rand_indx = 0, rand_input = None, change_rand = False, cmd='p'): #create_X_traces_with_rand_Nth_bit 
        #print(f"✔️ Starting measurement for {rand_indx}-th weights..")
        #cw_setuper.reset_target(scope) # flush chipwhisperer and re-assing its parameters (this is moot, if used >1x)
        rand_inputs = rand_input
        traces = []
        if rand_inputs is None:
            print(f"⚠️ Generating rand_input..")
            L = range((2**bnum)-1) # Range of generated rand_numbers
            rand_inputs = [random.choice(L) for _ in range(epochs)]
        input_array = [np.uint8(0) for i in range(input_size)]
        
        # capture X (epochs) traces, while each has different rand_input
        for i in trange(epochs, desc='                                Capturing traces'):
            input_array[rand_indx] = np.uint8(rand_inputs[i])
            input_data = bytearray(input_array)
            trace = capture_trace(input_data=input_data, cmd=cmd)
            traces.append(trace)
            
           # if i % 60 == 0:
           #     cw_setuper.reset_target(scope)
        return traces, rand_inputs

In [5]:
def save_data( waves, rand_inputs, out_data = None, name_extended = ""):
    import pandas as pd
    try:
        rand_inputs = pd.DataFrame(rand_inputs)
        rand_inputs.to_csv(f"database/rand_inputs_{name_extended}.csv")
        print("✔️ Saving Rand_input succesfull.")
    except Exception as e:
        print("❌ Saving Rand_input unsuccesfull because: ",e)
 
    files = [f"database/waves_{name_extended}.npy",f"database/rand_inputs_{name_extended}.csv"]
    np.save(files[0], waves)
    if out_data is not None:
        try:
            secret_weight = []
            for i in out_data[3:out_data[2]+2]:
                secret_weight.append(i)
            #df_weights  = pd.DataFrame(secret_weight)
            #df_weights.to_csv(f"database/weights_{name_extended}.csv")
            files.append(f"database/weights_{name_extended}.csv")
            np.save(files[2], np.array(secret_weight))
            print("🌍 Saving Secret_weights succesfull.")
        except Exception as e:
            print("❌ Saving Secret_weights unsuccesfull because: ",e)
    return files

In [ ]:
print("✔️ The ANN_Observer class succesfuly imported.")